In [22]:
import os
import random as rnd
import pandas as pd 

import pgutils as pg
import pgmath as pm
import algorithm as al
import numpy as np

In [6]:
#root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
root_dir = '../data/yes_small'

In [7]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])


In [8]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
train_dataset = pg.data_to_list(train_data[2:])

In [9]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)

In [10]:
# setting parameters
dimension = 2#rnd.choice([2, 5,10,25,50,100])
# regularization parameter set by cross validation
lam = rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
# regularization parameter for dual point model
nu = rnd.choice([0, lam])
# threshold for landmark dimension
r = 0.2
# number of landmark
n_landmarks = 50
# num iteration 100 o 200
n_iter = 4 #rnd.choice([100, 200])
# tau predefined learning rate
tau = 0.5

In [11]:
songs = len(song_hash)
transition_matrix = pg.transition_count(songs, train_dataset)
chunk = [[i for i in range(songs) if transition_matrix[s][i] > 0] for s in range(songs)]

In [52]:
print(chunk[0])

[1, 6, 10, 37, 38, 67, 78, 1670, 2781, 2828, 2830, 2837, 2839, 2874, 3056]


In [12]:
position = np.random.rand(songs, dimension)
num_transition = np.sum(transition_matrix)

params = pm.AlgParams(lam, nu, tau, num_transition, n_landmarks, r)

In [13]:
dim = len(position)
distance_mat = [[np.linalg.norm(position[i] - position[j]) for j in range(dim)] for i in range(dim)]
initial_distance = np.array(distance_mat).reshape((dim, dim))

In [14]:
print(np.array([len(chunk[9])]))
      

[40]


In [15]:
for i in range(len(chunk)):
    if np.min(np.array([len(chunk[i]) / songs ])) == 0:
        print(i)

1881
2015


In [16]:
landmarks = rnd.sample(range(songs), params.n_landmarks)
print(landmarks)

[2495, 862, 895, 878, 2928, 1399, 701, 2796, 1468, 1130, 154, 282, 2873, 1037, 1908, 1231, 1145, 390, 956, 2833, 1971, 2001, 37, 1573, 401, 2943, 469, 1889, 1440, 3017, 2021, 1768, 3056, 234, 1150, 2053, 3131, 2965, 2554, 2672, 2487, 136, 1628, 2540, 352, 131, 770, 570, 2884, 1938]


In [17]:
for s in range(songs):
    if len(chunk[s]) / songs < params.r:
        # mi sa che devo considerare non Z ma delta perché voglio la distanza tra le canzoni
        # Z indica la partition function
        closest_idx = np.argmin(np.array([initial_distance[s, j] for j in landmarks]))
        closest_landmark = landmarks[closest_idx]

        if closest_landmark not in chunk[s]:
            chunk[s].append(closest_landmark)

In [18]:
print(len(chunk[1881])/songs)
print(chunk[1881])

0.0003156565656565657
[2943]


In [19]:
print(position)

[[0.22673926 0.24642083]
 [0.70205014 0.13033557]
 [0.42985563 0.11996611]
 ...
 [0.71435821 0.2480803 ]
 [0.39302442 0.0914291 ]
 [0.31064869 0.77153135]]


In [23]:
batch = pg.initialize_landmarks(songs, params, position, transition_matrix)

KeyboardInterrupt: 

In [ ]:
#alleno l'algoritmo
U = al.single_point_algorithm(song_hash, dimension, train_dataset, n_iter, lam, nu, tau, n_landmarks, r)

In [ ]:
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
songs = len(song_hash)
n_test = np.sum(pg.transition_count(songs,test_dataset))
#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  